In [4]:
#import libraries
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

print('Libraries Imported.')

Libraries Imported.


In [119]:
# launch webdriver of target and gather html
url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

options = webdriver.ChromeOptions() 
# options.add_argument('--headless') 
browser = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                            options=options)
browser.get(url)

In [120]:
# navigate to product line tab
frameset = browser.find_element_by_tag_name('frameset')
print("Frameset: ", frameset)
frame = frameset.find_element_by_tag_name('frame')
frame_name = frame.get_attribute('name')
print(frame_name)
frame_suffix = frame_name[11:17]
print(frame_suffix)
print("Frame: ", frame.get_attribute('src'))

browser.switch_to.frame(frame)
product_line =  browser.find_element_by_id("Tab2")
# select product line tab
product_line.click()
# print(browser.page_source)

Frameset:  <selenium.webdriver.remote.webelement.WebElement (session="5c3ba1ba48c8f3d097aacb286ee36079", element="167f08a1-6634-44ec-aa5a-7e1951090c83")>
mainWindow_LOzXbe
LOzXbe
Frame:  https://echoinc.partsmartweb.com/scripts/EmpartISAPI.dll?MF&app=echo&session=1c3b193d-3f16-4cac-a0a4-8ec7e0acf78a&mode=12&TF=ewMain


In [121]:
prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
detailFrameID = 'plDetailFrame'
dataFrameName = 'ARICAT6TREE_' + frame_suffix

# print(prodLineFrameID)

frame = browser.find_element_by_id(prodLineFrameID)
browser.switch_to.frame(frame)

frame = browser.find_element_by_id(detailFrameID)
browser.switch_to.frame(frame)

frame = browser.find_element_by_name(dataFrameName)
browser.switch_to.frame(frame)

# click into ECHO assembly tree
cur_node = browser.find_element_by_id('pn18')
cur_node.click()

In [122]:
def getParts(nodeList):
    browser.implicitly_wait(2)

    # get all list items from unordered list param
    nodes = nodeList.find_elements_by_tag_name("li")

    # **on first run getParts is correctly gathering all li of first ul**
    # for node in nodes:
    #     print(node.get_attribute('id'))

    # iterate through all list items in current unordered list
    for node in nodes:
        print("List item ID: ", node.get_attribute('data-assembly-id'))
        # (base case) check if node is a parent, if false gather info from leaf node
        if node.get_attribute('data-assembly-is-parent') == "false":
            #gather data from part and pull to dataframe?
            print(' Reached leaf node.')
        # else: click() the element to expand child list
        else:
            # node_id = node.get_attribute('id')
            if node.get_attribute('aria-expanded') == "false":
                node.click()
            # node = browser.find_element_by_id(node_id)
            
            # browser.implicitly_wait(5)

            # ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
            # next_ul = WebDriverWait(browser, 5, ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.TAG_NAME, 'ul')))
            
            # print(node.get_attribute('outerHTML'))
            
            # find next unordered list element and call getParts recursively to iterate through next level
            getParts(node.find_element_by_tag_name("ul"))

In [123]:
# browser.implicitly_wait(3)
productList = browser.find_element_by_tag_name("ul")
getParts(productList)

List item ID:  19


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=93.0.4577.63)
